In [6]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math#
#import uproot
#from b2plot import hist
import b2plot as bp
import ROOT
#from root_pandas import read_root

import seaborn as sns
try:
    #plt.style.use('belle2')
    #plt.style.use('belle2_serif')
    plt.style.use('belle2_modern')
except OSError:
    print("Please install belle2 matplotlib style")   
    
print(plt.style.available)
px = 1/plt.rcParams['figure.dpi']

##############################################

def get_data(file='file', figname='name', tree='tree', base_filter=None,variables=None):
    
    tree=tree
    f = file
    base_filter  = base_filter    
    
    ROOT_df_start = ROOT.RDataFrame(tree, f)  
    col_dict  = ROOT_df_start.AsNumpy(variables)
    
    if base_filter !=None:
        ROOT_df_filtered  = ROOT_df_start.Filter(base_filter)                            
        col_dict  = ROOT_df_filtered.AsNumpy(variables)
    
    pd_df  = pd.DataFrame(col_dict)
    
    return pd_df
    
    

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'belle2', 'belle2_modern', 'belle2_serif', 'belle2_wg1', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [10]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, confusion_matrix

from mlxtend.plotting import plot_confusion_matrix

In [11]:
print(sys.version)

3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 07:04:59) [GCC 10.3.0]


In [12]:
variables=['M','cosHelicityAngleMomentum','daughter__bo1__cmE__bc','daughter__bo1__cmclusterTheta__bc','daughter__bo1__cmclusterNHits__bc',\
           'daughter__bo1__cmclusterE9E21__bc','daughter__bo1__cmminC2TDist__bc','daughter__bo1__cmclusterZernikeMVA__bc']

In [13]:
# file='/home/belle2/jaeyoung/workspace/06_MVA/my_pi0etaveto/train_v1/pi0veto_train.root'
file_pi0='/media/jykim/ssd_T5/storage/workspace/06_MVA/my_pi0etaveto/train_v1/pi0veto_train.root'



#file='/group/belle/users/jaeyoung/storage/pi0etaveto/frac_0_3/merged_frac_0_3/pi0etaveto_all_generic_recon_output.root'
# file2='/group/belle/users/jaeyoung/storage/pi0etaveto/signal/frac_0_5_a/half_1M_0_recon_mdst_all.root'
file_not_pi0='/media/jykim/ssd_T5/storage/pi0etaveto/signal/frac_0_5_a/half_1M_0_recon_mdst_all.root'


In [14]:
base_filter_pi0='isSignal==1'
# base_filter2='isSignal!=1 && daughter__bo0__cmgenMotherPDG__bc==421'
base_filter_not_pi0='isSignal!=1'

tree='pi0'
variables=['M','cosHelicityAngleMomentum','daughter__bo1__cmE__bc','daughter__bo1__cmclusterTheta__bc','daughter__bo1__cmclusterNHits__bc',\
           'daughter__bo1__cmclusterE9E21__bc','daughter__bo1__cmminC2TDist__bc','daughter__bo1__cmclusterZernikeMVA__bc']

pd_df_pi0 = get_data(file=file_pi0, tree=tree,variables=variables ,base_filter=base_filter_pi0)
pd_df_not_pi0 = get_data(file=file_not_pi0, tree=tree,variables=variables ,base_filter=base_filter_not_pi0)

In [15]:
pd_df_pi0.head()

,M,cosHelicityAngleMomentum,daughter__bo1__cmE__bc,daughter__bo1__cmclusterTheta__bc,daughter__bo1__cmclusterNHits__bc,daughter__bo1__cmclusterE9E21__bc,daughter__bo1__cmminC2TDist__bc,daughter__bo1__cmclusterZernikeMVA__bc
0,0.131333,-0.828418,0.075256,1.253214,1.953125,1.000000,50.781250,0.461914
1,0.135979,-0.938767,0.046253,2.131419,2.929688,1.000000,138.671875,0.537109
2,0.133513,-0.465027,0.259318,1.868852,9.960938,0.976562,80.322266,0.291992
3,0.112952,-0.457962,0.263096,0.421573,7.031250,0.994141,20.996094,0.766602
4,0.118118,-0.095041,1.077864,0.597266,8.984375,0.999023,212.890625,0.795898


In [16]:
pd_df_not_pi0.head()

,M,cosHelicityAngleMomentum,daughter__bo1__cmE__bc,daughter__bo1__cmclusterTheta__bc,daughter__bo1__cmclusterNHits__bc,daughter__bo1__cmclusterE9E21__bc,daughter__bo1__cmminC2TDist__bc,daughter__bo1__cmclusterZernikeMVA__bc
0,0.504665,-0.750407,0.180713,0.419016,8.789062,0.961914,74.218750,0.058594
1,0.375794,-0.916924,0.097773,0.435058,5.078125,0.991211,60.791016,0.058594
2,0.368903,-0.759519,0.120542,0.808697,8.007812,0.860352,102.539062,0.087891
3,0.180046,-0.739156,0.075909,1.592156,5.078125,1.000000,30.273438,0.721680
4,0.491069,-0.144743,0.328259,1.882007,8.007812,0.988281,231.445312,0.907227


In [ ]:
# 모델 학습
model = HistGradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3,random_state=42)
model.fit(learning_train_data, train_labels)